# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.19,87,45,1.79,EC,1692497555
1,1,adamstown,-25.0660,-130.1015,17.24,53,3,9.24,PN,1692497277
2,2,albany,42.6001,-73.9662,18.23,73,4,3.03,US,1692497270
3,3,coquimbo,-29.9533,-71.3436,15.85,72,20,3.09,CL,1692496875
4,4,waitangi,-43.9535,-176.5597,12.12,95,100,4.92,NZ,1692497336


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
# %%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    hover_cols=['City'], 
    alpha=0.5,
    geo=True,
    title='City Map with Humidity',
    tiles= "OSM"
)

# Display the map
scatter_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Define your ideal weather conditions
ideal_conditions = (
    (city_data_df['Max Temp'] > 21) &  # Max Temp higher than 21°C
    (city_data_df['Max Temp'] < 27) &  # Max Temp lower than 27°C
    (city_data_df['Wind Speed'] < 4.5) &  # Wind speed less than 4.5 m/s
    (city_data_df['Cloudiness'] == 0)    # Cloudiness is zero
)

# Apply the filter to the DataFrame
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,touros,-5.1989,-35.4608,24.03,87,0,3.76,BR,1692497332
77,77,hamilton,39.1834,-84.5333,24.43,69,0,4.12,US,1692497260
103,103,lompoc,34.6391,-120.4579,24.45,75,0,3.60,US,1692497371
148,148,vernon,50.2581,-119.2691,24.31,30,0,1.54,CA,1692497615
152,152,paradise point,-27.8863,153.3957,24.37,45,0,1.79,AU,1692497617


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
63,touros,BR,-5.1989,-35.4608,87,
77,hamilton,US,39.1834,-84.5333,69,
103,lompoc,US,34.6391,-120.4579,75,
148,vernon,CA,50.2581,-119.2691,30,
152,paradise point,AU,-27.8863,153.3957,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
        "categories": "accommodation.hotel", 
        "limit": 1,  
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display updated data
hotel_df




touros - nearest hotel: Pousada Atlântico
hamilton - nearest hotel: North Vista Manor
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
vernon - nearest hotel: Okanagan Royal Park Inn
paradise point - nearest hotel: Hope Island Resort
gignac - nearest hotel: Le Vieux Moulin
ioannina - nearest hotel: Παλλάδιο
humberto de campos - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
fort bragg - nearest hotel: Airborne Inn Lodging
chulakkurgan - nearest hotel: адеми
urumqi - nearest hotel: 百时快捷酒店红山店
carnarvon - nearest hotel: No hotel found
navrongo - nearest hotel: Mayaga
kurtalan - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة-سرت
at taj - nearest hotel: No hotel found
filadelfia - nearest hotel: The Ritz-Carlton, Philadelphia
remire-montjoly - nearest hotel: Complexe Belova
siracusa - nearest hotel: Hotel Panorama
misratah - nearest hotel: الفندق السياحى
thomaston - nearest hotel: Quality Inn
awjilah - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
63,touros,BR,-5.1989,-35.4608,87,Pousada Atlântico
77,hamilton,US,39.1834,-84.5333,69,North Vista Manor
103,lompoc,US,34.6391,-120.4579,75,Embassy Suites by Hilton Lompoc Central Coast
148,vernon,CA,50.2581,-119.2691,30,Okanagan Royal Park Inn
152,paradise point,AU,-27.8863,153.3957,45,Hope Island Resort
164,gignac,FR,43.6520,3.5509,48,Le Vieux Moulin
179,ioannina,GR,39.6675,20.8508,69,Παλλάδιο
197,humberto de campos,BR,-2.5983,-43.4611,84,No hotel found
201,colonia,DE,50.9333,6.9500,88,Wasserturm Hotel Cologne
232,fort bragg,US,35.1390,-79.0060,75,Airborne Inn Lodging


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Hotel Name', 
    hover_cols=['City', 'Hotel Name', 'Country'],
    geo=True,
    alpha=0.7,
    tiles="OSM"
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)